In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import panel as pn
pn.extension("ipywidgets", 'katex', 'mathjax')
import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
#from matplotlib.ticker import MultipleLocator
from matplotlib.figure import Figure

from random import shuffle, uniform

import sys
from inspect import signature
print("Packages succesfully loaded")

# Functions from cookbook

# Commonly used functions

In [ ]:
def wave_length(T, h):
    L = 9.81 * T**2 / (2 * np.pi)
    L_all = [L]

    for i in range(1500):
        L = 9.81 * T**2 / (2 * np.pi) * np.tanh(2 * np.pi * h / L)
        L_all.append(L)

        # stop the iteration when the error is sufficiently small
        if np.abs(L_all[-1] - L_all[-2]) < 0.0005:
            break
            
    return round(L, 13)

# Questions

In [ ]:
def W3_interactiveplot_normal_indicent_waves(H0, T, d0, slope_in):
    
    slope = 1.0 / slope_in         # bed slope [-]
    d0                             # offshore water depth [m]
    x_max = round((d0+2)/slope)
    x = np.arange(0, x_max + 1, 1) # cross-shore coordinate [m]
    zbed = -(d0 - slope * x)       # bed elevation [m]
    h = -zbed                      # still water depth [m]
    h[h < 0] = 0                   # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]
    
    # given:
    gamma = 0.8               # wave breaking ratio
    
    # The wave characteristics at every location in the cross-section
    L = np.array([wave_length(T, h) for h in h])  # The wave length
    c = L/T                                       # The wave celerity
    k = 2*np.pi/L                                 # The wave number
    n = 0.5 + (k*h/np.sinh(2*k*h))                
    cg = n*c                                      # The wave group celerity
    Ksh = np.sqrt(cg[0]/cg)                       # The shoaling parameter  
    H = H0*Ksh                                    # The wave height due to shoaling (only)
    H_shoal = H.copy()
    Hbreaking = gamma * h                         # The wave-breaking height
    H[H>Hbreaking]=Hbreaking[H>Hbreaking]         # The wave height

    fig, axs = plt.subplots(nrows = 4, ncols = 2, figsize = (9,6), sharex=True, sharey = False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)
   
    axs[0,0].plot(x, zbed, label="Bed level [m] (1:" + str(round(slope_in,2)) + ')', color="k")
    axs[0,0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]")  
    axs[1,0].plot(x_water, L, label= 'Wavelength (L) [m]', color="k")
    axs[2,0].plot(x_water, k, label= 'Wave number (k) [rad/m]', color="k")
    axs[3,0].plot(x_water, c, label= 'Wave celerity (c) [m]', color="k")
    axs[0,1].plot(x_water, cg, label= 'Wavegroup celerity (cg) [m/s]', color="k")    
    axs[1,1].plot(x_water, Ksh, label= 'shoaling factor (ksh) [-]', color="k")
    axs[2,1].plot(x_water, H_shoal, label= 'shoaling wave height [m]', color="k")
    axs[3,1].plot(x_water, H, label= 'breaking wave height [m]', color="k")
    
    for ax in axs:
        ax[0].legend(loc="lower left")
        ax[0].xaxis.set_visible(False)
        ax[1].legend(loc="best")
        ax[1].xaxis.set_visible(False)
    axs[3,0].xaxis.set_visible(True)
    axs[3,1].xaxis.set_visible(True)

    
def W3_plot_normal_indicent_waves():
    # Create interactive widgets, which require IPY Widgets, widgets from panel do not work
    #H0 = pn.widgets.FloatInput(value=1.5, start=0, end=500, step=0.1, name="Offshore wave height (H0) [m]")
    #T = pn.widgets.FloatInput(value=5, start=0.05, end=500, step=0.01, name ="Wave period (T) [s]")
    #slope = pn.widgets.FloatInput(value=30, start=0.1, end=50, step=0.1, name ="slope 1:...")
    #d0 = pn.widgets.FloatInput(value=50, start=0.1, end =500, step=0.1, name="offshore depth (h0) [m]")


    H0 = ipw.FloatText(value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width = 75)
    T = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description ="T [s]")
    slope = ipw.FloatText(value=30, min=0.1, max=50, step=0.1, description ="slope 1:...")
    d0 = ipw.FloatText(value=25, min=0.1, max =500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox([H0, T])
    Vbox2 = ipw.VBox([d0, slope])
    
    widgets = ipw.HBox([Vbox1, Vbox2])
    graph = ipw.interactive_output(W3_interactiveplot_normal_indicent_waves, {'H0': H0,'T':T, 'd0': d0, 'slope_in': slope})
    
    display(widgets, graph)

W3_plot_normal_indicent_waves()

In [ ]:
1) Which method is used to calculate the wave length? --> iterative approach
2) Various questions on the influence of H when parameters change.

Building for cookbook, having a number of multiple-selection questions, with one score.
todo:
- change widget to multiple selection
- calculate score, up to 1 pt per question.

In [ ]:
def multiple_selection_questions(questions, choices, answer_ids):
    # define the widgets for visualization, make for each a row with a question and answer
    # Store all the rows with statements in a list for visualization, and in toggle_widgets for checking the answer
    Rows = []
    toggle_widgets = []
    for i in range(len(choices)):
        question_widget = pn.widgets.StaticText(value=questions[i], width = 150)  #statement
        
        radio_group_widget = pn.widgets.RadioButtonGroup(name='Radio Button Group', options=choices[i], button_type='default')
        toggle_widgets.append(radio_group_widget)
        
        add_row = pn.Row(question_widget, radio_group_widget)
        Rows.append(add_row)

    # randomize the order of statements
    shuffle(Rows)

    # Add a submit button with a feedback option next to it
    submit_button =  pn.widgets.Button(name="Check")
    feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
    submit_row = pn.Row(submit_button, feedback_widget)
    
    UI = *Rows, submit_row

    # check the answer and give feedback
    def check_answers(button):
        score = 0

        for i in range(len(questions)):
            for j in range (len(answer_ids[i])):
                print(answer_ids[i][j])
            #if toggle_widgets[i].value == answers[correct_answers_id[i]]:
            #    score += 1

        # print(toggle_widget.value)
        #feedback_widget.value = "Your score is " + str(score) + "/" + str(len(correct_answers_id))

    submit_button.on_click(check_answers)

    return UI

def ask_select_statement():

    # define the questions and store them in a list
    question_1 = 'Question 1'
    choices_1 = ["good", "wrong", "false"]
    answer_id_1 = [0]
    
    question_2 = 'Question 2'
    choices_2 = ["good", "good", "wrong"]
    answer_id_2 = [0,1]
    
    question_3 = 'Question 3'
    choices_3 = ["You should select this answer", "this answer is wrong", "This answer is also incorrect"]
    answer_id_3 = [0]
    
    questions = [question_1, question_2, question_3]
    choices = [choices_1, choices_2, choices_3]
    answer_ids = [answer_id_1,answer_id_2,answer_id_3]

    # Add the text of the question on top
    text_general = "Select if the wave described on the right experiences shallow"
    
    # make the widgets and display them
    text_widget = pn.widgets.StaticText(value=text_general)
    question_widgets = multiple_selection_questions(questions, choices, answer_ids)
    display(text_widget, *question_widgets)


ask_select_statement()